# Description

Run the code to create the queries for every profile id and store it in a BQ table.
Fill in and check all the variables in the 2nd and 3rd cell before running the notebook.
The results will be appended to the desired destination table


In [ ]:
import urllib.request
import pandas as pd
import pandas_gbq
from google.cloud import bigquery, storage
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
from google.colab import auth

In [ ]:
PROJECT_ID = "" # Destination project where the query ids will be stored
DATASET_ID = "" # Destination dataset where the query ids will be stored
TABLE_ID = "" # Destination table where the query ids will be stored
QUERY_TITLE_PREFIX = "" # A prefix to diffrentiate between querys ids
EMAILS = [''] # List of emails which will receive DV360 notifications
TYPE = "" # See DV360 API docs.
START_YEAR = 2022
START_MONTH = 7
START_DAY = 1
END_YEAR = 2022
END_MONTH = 7
END_DAY = 1
DIMENSIONS = []
METRICS = []

bucket_name = '' # bucket name where the credentials for the service account are stored
credentials_path = '' # Path to credentials. i.e.: 'credentials/creds.json'
api_name = 'doubleclickbidmanager'
api_version = 'v2'
SCOPES = 'https://www.googleapis.com/auth/doubleclickbidmanager'
partner_ids = [] # List of partners ids (int)

In [ ]:
request_body = {
    "metadata": {
        "title": "",
        "dataRange": {
            "range": "CUSTOM_DATES",
            "customStartDate": {
                "year": START_YEAR,
                "month":START_MONTH,
                "day": START_DAY
            },
            "customEndDate": {
                "year": END_YEAR,
                "month":END_MONTH,
                "day": END_DAY
            }
        },
        "format": "CSV",
        "sendNotification": True,
        "shareEmailAddress": EMAILS
    },
    "params": {
        "type": TYPE,
        "groupBys": DIMENSIONS,
        "metrics": METRICS,
        "filters": [
            {
                "type": "FILTER_PARTNER",
                "value": ""
            }
        ]
    },
    "schedule": {
        "frequency": "ONE_TIME"
    }
}

In [ ]:
# Authenticate the user to access BigQuery and GCS
auth.authenticate_user()
bq_client = bigquery.Client()
storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_name)

# Create BQ table
schema = [
    bigquery.SchemaField("report_name", "STRING"),
    bigquery.SchemaField("partner_id", "STRING"),
    bigquery.SchemaField("title", "STRING"),
    bigquery.SchemaField("query_id", "STRING")
]
#table = bigquery.Table(f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}", schema=schema)
#table = bq_client.create_table(table)  # Make an API request to create table
table = bigquery.Table(f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}") # Get table

# Make connection with DV360
# Get credentials from bucket currently says bucket in service
credentials_blob = bucket.blob(credentials_path)
with open('/tmp/credentials.json', 'wb') as file_obj:
    credentials_blob.download_to_file(file_obj)
service_account_credentials = ServiceAccountCredentials.from_json_keyfile_name("/tmp/credentials.json", scopes=SCOPES)

# Build the service object to connect to DV360 API
service = build(api_name, api_version, cache_discovery=False,credentials=service_account_credentials)

In [ ]:
data = []
for partner in partner_ids:
    # Set queries params
    request_body['metadata']['title'] = f"{QUERY_TITLE_PREFIX}_{partner}"
    request_body['params']['filters'][0]['value'] = str(partner)
    # Create Query
    create_query_response = service.queries().create(body=request_body).execute()
    query_id = create_query_response['queryId']
    data.append(
        {
            'report_name':QUERY_TITLE_PREFIX,
            'partner_id':str(partner),
            'title':f"{QUERY_TITLE_PREFIX}_{partner}",
            'query_id':str(query_id),
        }
    )

query_ids_df = pd.DataFrame(data)
pgbq_response = pandas_gbq.to_gbq(query_ids_df, f"{DATASET_ID}.{TABLE_ID}", project_id=PROJECT_ID, if_exists="append")
pgbq_response

1it [00:04,  4.89s/it]
